In [1]:
import os
import scipy
import scipy.io as scio
import numpy as np

In [2]:
# from imblearn.over_sampling import SMOTE 
# sm = SMOTE(random_state=42)
from imblearn.combine import SMOTEENN
sm = SMOTEENN(random_state=42)


In [3]:
# 进行随机下采样，并使用MLP分类
import scipy
import scipy.io as scio
import numpy as np
import os
# 读取训练和测试数据
data_root = '/data/file/classification_data/2012-2017/'
sumX = []
sumy = []
for year in range(2012, 2017):
    print(year)
    trainX_norm = scio.loadmat(os.path.join(data_root, 'data'+str(year)+'.mat'))['X']
    trainy_norm = scio.loadmat(os.path.join(data_root, 'label'+str(year)+'.mat'))['y'].reshape(-1, 1)

    print('trainy_norm.shape:', trainy_norm.shape)
    print('data'+str(year)+'.mat')

    X_re, y_re = sm.fit_resample(trainX_norm, trainy_norm)

    sumX.append(X_re)
    sumy.append(y_re)
data_train = np.concatenate(sumX, axis=0)  # 将列表中的数据按0轴合并
label_train = np.concatenate(sumy, axis=0)

testX = scio.loadmat(os.path.join(data_root, 'testdata.mat'))['X']
testy = scio.loadmat(os.path.join(data_root, 'testlabel.mat'))['y'].reshape(-1, 1)

2012
trainy_norm.shape: (1447, 1)
data2012.mat
2013
trainy_norm.shape: (1251, 1)
data2013.mat
2014
trainy_norm.shape: (1313, 1)
data2014.mat
2015
trainy_norm.shape: (1403, 1)
data2015.mat
2016
trainy_norm.shape: (1817, 1)
data2016.mat


In [4]:
test_X_re, test_y_re = sm.fit_resample(testX, testy)

In [5]:

print(data_train.shape)
print(label_train.shape)
print(testX.shape)
print(testy.shape)
print(test_X_re.shape)
print(test_y_re.shape)


(14244, 448)
(14244,)
(2071, 448)
(2071, 1)
(4030, 448)
(4030,)


In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(data_train, label_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [7]:
clf.score(testX, testy)

0.8348623853211009

In [8]:
y_pred = clf.predict(testX)

In [9]:
clf.score(test_X_re, test_y_re)

0.6672456575682382

In [10]:
# f1
from sklearn.metrics import f1_score
print(f1_score(testy, y_pred, average='weighted'))
print(f1_score(testy, y_pred, average='macro'))
print(f1_score(testy, y_pred, average='micro'))

0.887450013903694
0.511467662103329
0.8348623853211009


In [11]:
# 混淆矩阵
y_pred_re = clf.predict(test_X_re)
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y_re, y_pred_re)

array([[1707,  308],
       [1033,  982]])

In [12]:
# auc
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(test_y_re, y_pred_re)
metrics.auc(fpr, tpr)

0.6672456575682381